In [8]:
!pip install datasets transformers torch accelerate pandas huggingface-hub


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [9]:
import torch
import pandas as pd
from transformers import pipeline
from datasets import load_dataset
from tqdm.auto import tqdm
from huggingface_hub import notebook_login

MODEL_NAME = "iPad7/kcbert-base-sentiment-0.1b"
DATASET_ID = "iPad7/SKN13-3rd-3Team-datasets"

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
sentiment_pipeline = pipeline(
    "text-classification",
    model=MODEL_NAME,
    device=device
)

dataset = load_dataset(DATASET_ID, split='train')
print(dataset)

cuda


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

...

data/train-00006-of-00009-f1f4fde8276506(…):   0%|          | 0.00/20.2M [00:00<?, ?B/s]

data/train-00007-of-00009-144e1c817f05b7(…):   0%|          | 0.00/20.2M [00:00<?, ?B/s]

data/train-00008-of-00009-169b4192040c05(…):   0%|          | 0.00/20.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3092010 [00:00<?, ? examples/s]

Dataset({
    features: ['제품명', '성분', '별점', '피부타입', '피부고민', '자극도', '카테고리', '리뷰'],
    num_rows: 3092010
})


In [6]:
def predict_sentiment(batch) -> list:
    predictions = sentiment_pipeline(batch['리뷰'], truncation=True, batch_size=16)
    labels = [p['label'] for p in predictions]
    batch['리뷰_감성'] = labels
    return batch

updated_dataset = dataset.map(
    predict_sentiment, 
    batched=True, 
    batch_size=32
)

print(updated_dataset)

Map:   0%|          | 0/3092010 [00:00<?, ? examples/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Dataset({
    features: ['제품명', '성분', '별점', '피부타입', '피부고민', '자극도', '카테고리', '리뷰', '리뷰_감성'],
    num_rows: 3092010
})


In [11]:
result_df = pd.DataFrame(updated_dataset)
result_df.sample()

,제품명,성분,별점,피부타입,피부고민,자극도,카테고리,리뷰,리뷰_감성
450089,라운드랩 포 맨 소나무 진정 시카 크림,"정제수,부틸렌글라이콜,글리세린,카프릴릭/카프릭트라이글리세라이드,1,2-헥산다이올,하...",5,지성에 좋아요,진정에 좋아요,자극없이 순해요,크림,순해서 너무 좋고 트러블도 많이 줄었어요.,긍정


In [12]:
notebook_login()

In [14]:
updated_dataset.push_to_hub('iPad7/SKN13-3rd-3Team-labeled-datasets')

Uploading the dataset shards:   0%|          | 0/9 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/344 [00:00<?, ?ba/s]

Uploading...:   0%|          | 0.00/20.7M [00:00<?, ?B/s]

Creating parquet from Arrow format:   0%|          | 0/344 [00:00<?, ?ba/s]

Uploading...:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

...

Creating parquet from Arrow format:   0%|          | 0/344 [00:00<?, ?ba/s]

Uploading...:   0%|          | 0.00/20.4M [00:00<?, ?B/s]

Creating parquet from Arrow format:   0%|          | 0/344 [00:00<?, ?ba/s]

Uploading...:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/iPad7/SKN13-3rd-3Team-labeled-datasets/commit/07b71d09aa05ef76b0bb53a96be0d1efbc85f7d9', commit_message='Upload dataset', commit_description='', oid='07b71d09aa05ef76b0bb53a96be0d1efbc85f7d9', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/iPad7/SKN13-3rd-3Team-labeled-datasets', endpoint='https://huggingface.co', repo_type='dataset', repo_id='iPad7/SKN13-3rd-3Team-labeled-datasets'), pr_revision=None, pr_num=None)